# Factory Methodパターン
*  オブジェクトの作成をサブクラスに委ねることで，利用者側（クライアント／別のシステム）を具体的なクラスのインスタンス化から独立させる
*  オブジェクトの生成とオブジェクトの具体的な処理（実装）を分離することで（単一責任の原則遵守），柔軟にオブジェクトを利用でき再利用性を高める  
*  直感的なイメージ:
>*  製品（オブジェクト）を生成する工場（クラス）のテンプレートがある
>*  具体的な工場（車工場や船工場）はこのテンプレートに基づいて作られる
>*  これらの具体的な工場から製品が生成される
*  具体的には，インターフェースで処理の骨組みを作り，サブクラスを用いてオブジェクトを作成する
*  サブクラスに応じて，作成されるオブジェクトのタイプを変える 


## Factory Methodパターンの構成要素
*  Product:
>*  生成されるオブジェクトの構成要素を定義するインターフェースまたは抽象クラス
>*  具体的な構成要素は, サブクラス（具象クラス）のConcreteProductで定義
*  ConcreteProduct:
>*  Productインターフェースを実装する具体的なクラス（複数作成）
*  Creator:
>*  Productのオブジェクト（インスタンス）を生成する処理を定義したインターフェースまたは抽象クラス
>*  具体的な生成処理（`create`）については, サブクラス（具象クラス）のConcreteCreatorで定義
*  ConcreteCreator:
>*  Creatorクラスを継承し，Creatorを具体化するクラス
>*  ConcreteProductのオブジェクト（インスタンス）を生成する具体的な処理を定義したクラス


> <img src='./fig/FactoryMethod_templete.jpg' width='750'>

## Factory Methodパターンの例 その1
*  以下のコードでは，異なる種類の乗り物オブジェクト（車と船）を作成するためのファクトリーメソッドパターンを実装している
*  `Factory`クラス（インターフェース）はCreatorで，抽象メソッド`create`を持つ
*  `create`メソッドは，乗り物のオーナーの名前を引数として受け取る
*  ConcreteCreatorである`CarFactory`クラスと`ShipFactory`クラスは，`Factory`に対する具象クラスとして`create`メソッドの実装をそれぞれ定義する
*  実装されたメソッドは，ConcreteProductである`Car`クラスのインスタンスと`Ship`クラスのインスタンスをそれぞれ生成する
*  `Car`クラスと`Ship`クラスのスーパークラスが抽象クラスの`Product`で，抽象メソッド`use`を持つ
*  `use`メソッドの実装は，`Car`クラスと`Ship`クラスでそれぞれ定義する
*  この例のように，Factory Methodパターンは，具体的な乗り物オブジェクトの生成を利用者（クライアント／他のシステム）から隠蔽し，動的にオブジェクトを生成している
*  これにより，柔軟かつ拡張可能な設計を実現する
*  Factory Methodパターンは，オブジェクトの作成プロセスが複雑な場合や，利用者側で利用するオブジェクトが属するクラスの詳細を知らなくてもよい場合に特に有効に働く

> <img src='./fig/design_patterns_factorymethod_example.jpg' width='800'>


In [ ]:
from abc import ABC, abstractmethod

# Product
class Product(ABC):

    def __init__(self, owner: str) -> None:
        self._owner = owner

    @abstractmethod
    def use(self) -> None:
        pass

# ConcreteProduct
class Car(Product):

    def use(self) -> None:
        print(f'{self._owner} is driving the car.')

# ConcreteProduct
class Ship(Product):

    def use(self) -> None:
        print(f'{self._owner} is driving the ship.')

# Creator
class Factory(ABC):

    @abstractmethod
    def create(self, owner: str) -> Product:
        pass

# ConcreteCreator
class CarFactory(Factory):

    def create(self, owner: str) -> Car:
        return Car(owner)

# ConcreteCreator
class ShipFactory(Factory):

    def create(self, owner: str) -> Ship:
        return Ship(owner)

# クライアント
def client_code(factory: Factory, owner: str) -> None:
    product: Product = factory.create(owner)
    product.use()

car_factory: Factory = CarFactory()
ship_factory: Factory = ShipFactory()

client_code(car_factory, 'Alice')
client_code(car_factory, 'Bob')

client_code(ship_factory, 'John')
client_code(ship_factory, 'Paul')

Alice is driving the car.
Bob is driving the car.
John is driving the ship.
Paul is driving the ship.


## Factory Methodパターンの例 その2
* ここでは，LSP（リスコフの置換原則）に違反しているコードを，Factory Methodパターンを使ってOCP遵守のコードに改善する例を考える
* LSPについては，[第10回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook10.ipynb)を参照



### LSPに違反している例
* 以下のコードは，音楽プレイヤーをイメージしたコードになっている（下図はそのクラス図）
* このコードにおいて，スーパークラスとなる `MusicPlayer` で「プレーヤーを使って音楽を再生する」という契約を結んでいる
* この契約は，`play`メソッドに対応しており，引数`file`として再生するファイル（`str`）を受け取る形になっている
* しかし，サブクラスは上記契約を破っている ⇒ LSP違反
>* `Mp3Player` は「拡張子が .mp3 であること」を追加前提としている
>* `WavPlayer` はさらに「`driver_loaded` が `True`（WAVドライバ読み込み済み）」 という前提まで要求している
* ドライバが読み込まれてない場合（`driver_loaded`が`False`）は，OSレベルの操作（ファイルや入出力）で失敗したときに起こる例外である OSError を発生させている

> <img src='./fig/factorymethod_player1.jpg' width='420'>

**コードに潜在する問題:**
* クライアントは「再生失敗時に行う共通の処理」を前提にできない
* `WavPlayer` がデフォルトで再生不能な状態（`driver_loaded: bool=False`） ⇒ クライアントは，常に音楽が再生できるという期待を持てない


**【補足】`if`文の条件式で用いている `endswith` について:**
* 文字列（`str`）に対するメソッドの一つ
* 引数で指定した文字列（接尾辞）で終わっているかどうかを調べる
* 戻り値は `bool` で，指定した文字列で終わっていれば `True`，それ以外は `False` を返す





In [ ]:
from abc import ABC, abstractmethod

# スーパークラス
class MusicPlayer(ABC):
    def play(self, file: str) -> None:
        pass

# サブクラス
class Mp3Player(MusicPlayer):
    def play(self, file: str) -> None:
        if not file.endswith('.mp3'):
            raise ValueError('mp3ファイルだけ再生できます')
        print(f'[MP3] {file} を再生しました')

# サブクラス
class WavPlayer(MusicPlayer):
    def __init__(self, driver_loaded: bool = False) -> None:
        self.driver_loaded = driver_loaded

    def play(self, file: str) -> None:
        if not self.driver_loaded:
            raise OSError('WAVドライバが読み込まれていません')
        if not file.endswith('.wav'):
            raise ValueError('wavファイルだけ再生できます')
        print(f'[WAV] {file} を再生しました')


# クライアント
def client_code(player: MusicPlayer, file: str):
    player.play(file)

client_code(Mp3Player(), 'song.mp3')
client_code(WavPlayer(), 'song.wav') # 例外発生

### LSPを遵守したコードに改善
* ここで，Factory Methodパターンを使ってLSPを遵守したコードに改善する
* 改善したコードは以下のとおりで，クラス図は下図のように描ける

> <img src="./fig/factorymethod_player2.jpg" width="580">

**Factory Methodパターンの構成要素:**
>* Creator: `MusicApp`クラス
>* ConcreteCreator: `DefaultMusicApp`クラス
>* Product: `MusicPlayer`クラス
>* ConcreteProduct: `Mp3Player`クラスと`WavPlayer`クラス

**Productの生成について:**
*  `create`メソッドは，再生するファイル名を引数として受け取る
*  `create`メソッドは，`play_music`メソッドの中で呼び出される ⇒ ファイル再生時に，ファイルの種類に応じて再生可能なプレイヤーを生成する
*  つまり，Productを Creator 経由で適切に選択・生成する

**失敗時の処理について:**
*  独自の例外「PlaybackError」を定義し，再生失敗時にこの例外を発生させる ⇒ 失敗時の共通の処理パターンが確立できる
*  これにより，クライアントは「失敗時は PlaybackError を捕捉すればよい」という期待で `MusicPlayer` を利用できる
*  また，`WavPlayer` はコンストラクタ内で必ずドライバを読み込んでいるので，`WavPlayer`自身で条件を満たすことができる

In [ ]:
from abc import ABC, abstractmethod

# 独自の例外: 失敗時に発生
class PlaybackError(Exception):
    pass

# Product
class MusicPlayer(ABC):
    @abstractmethod
    def play(self, file: str) -> None:
        pass

# ConcreteProduct
class Mp3Player(MusicPlayer):
    def play(self, file: str) -> None:
        if not file.endswith('.mp3'):
            raise PlaybackError('unsupported format')
        print(f'[MP3] {file} を再生しました')

# ConcreteProduct
class WavPlayer(MusicPlayer):
    def __init__(self) -> None:
        print('WAVドライバを読み込みました') # ここで必ずロードして再生条件を満たす
        self._driver_loaded: bool = True

    def play(self, file: str) -> None:
        if not self._driver_loaded:
            raise PlaybackError('driver not loaded')
        if not file.endswith('.wav'):
            raise PlaybackError('unsupported format')
        print(f'[WAV] {file} を再生しました')

# Creator
class MusicApp(ABC):
    @abstractmethod
    def create(self, file: str) -> MusicPlayer:
        pass

    def play_music(self, file: str) -> None:
        player = self.create(file)
        player.play(file)

# ConcreteCreator
class DefaultMusicApp(MusicApp):
    def create(self, file: str) -> MusicPlayer:
        if file.endswith('.mp3'):
            print('MP3プレイヤーを起動します')
            return Mp3Player()
        if file.endswith('.wav'):
            print('WAVプレイヤーを起動します')
            return WavPlayer()
        raise PlaybackError('未対応の音楽形式です')

# クライアント
def client_code(app: MusicApp, file: str) -> None:
    app.play_music(file)

app: MusicApp = DefaultMusicApp()
client_code(app, 'song.mp3')
client_code(app, 'song.wav')

MP3プレイヤーを起動します
[MP3] song.mp3 を再生しました
WAVEプレイヤーを起動します
WAVドライバを読み込みました
[WAV] song.wav を再生しました


### コードの拡張
* 上記のコードの拡張を考える
* ConcreteProductの追加: FLAC用プレイヤー `FlacPlayer` を追加する
* ConcreteCreatorの追加: 常に`Mp3Player`だけを生成する `Mp3OnlyApp` を追加する（軽量アプリのようなイメージ）
* 追加・修正した点
>* `FlacPlayer`の定義
>* `FlacPlayer`のインスタンス生成のために `DefaultMusicApp` に`if`文を追加
>* `Mp3OnlyApp`の定義
* 他の部分は変更なし

In [ ]:
from abc import ABC, abstractmethod

# 独自の例外: 失敗時に発生
class PlaybackError(Exception):
    pass

# Product
class MusicPlayer(ABC):
    @abstractmethod
    def play(self, file: str) -> None:
        pass

# ConcreteProduct
class Mp3Player(MusicPlayer):
    def play(self, file: str) -> None:
        if not file.endswith('.mp3'):
            raise PlaybackError('unsupported format')
        print(f'[MP3] {file} を再生しました')

# ConcreteProduct
class WavPlayer(MusicPlayer):
    def __init__(self) -> None:
        print('WAVドライバを読み込みました') # ここで必ずロードして再生条件を満たす
        self._driver_loaded: bool = True

    def play(self, file: str) -> None:
        if not self._driver_loaded:
            raise PlaybackError('driver not loaded')
        if not file.endswith('.wav'):
            raise PlaybackError('unsupported format')
        print(f'[WAV] {file} を再生しました')

# ConcreteProduct（新規追加）
class FlacPlayer(MusicPlayer):
    def play(self, file: str) -> None:
        if not file.endswith(".flac"):
            raise PlaybackError('unsupported format')
        print(f"[FLAC] {file} を再生しました")

# Creator
class MusicApp(ABC):
    @abstractmethod
    def create(self, file: str) -> MusicPlayer:
        pass

    def play_music(self, file: str) -> None:
        player = self.create(file)
        player.play(file)

# ConcreteCreator
class DefaultMusicApp(MusicApp):
    def create(self, file: str) -> MusicPlayer:
        if file.endswith('.mp3'):
            print('MP3プレイヤーを起動します')
            return Mp3Player()
        if file.endswith('.wav'):
            print('WAVプレイヤーを起動します')
            return WavPlayer()
        # FlacPlayerを生成するためのif文追加
        if file.endswith('.flac'):
            print('FLACプレイヤーを起動します')
            return FlacPlayer()
        raise PlaybackError('未対応の音楽形式です')

# ConcreteCreator（新規追加）
class Mp3OnlyApp(MusicApp):
    def create(self, file: str) -> MusicPlayer:
        return Mp3Player()

# クライアント
def client_code(app: MusicApp, file: str) -> None:
    app.play_music(file)

app1: MusicApp = DefaultMusicApp()
client_code(app1, 'song.mp3')
client_code(app1, 'song.wav')
client_code(app1, 'song.flac') # FLACファイルの再生

app2: MusicApp = Mp3OnlyApp()
client_code(app2, 'song.mp3')
client_code(app2, 'song.wav') # 例外発生


## Factory Methodパターンのメリット・デメリット

| 観点         | メリット                                          | デメリット                                 |
| ---------- | --------------------------------------------- | ------------------------------------- |
| **拡張性**    | 新しい製品を追加するときは，新しいサブクラスを追加するだけで済む（OCP遵守）   | 製品が増えるたびにサブクラスが増えてクラス数が多くなる      |
| **責任分離**   | 「製品を作る部分」と「製品を使う部分」を分けられる（SRP遵守）           | クラスの関係が間接的になるため，初心者には理解が難しい           |
| **柔軟性**    | 実行時に「どの ConcreteCreator を使うか」を切り替えることで，生成物を動的に選択可能 | Creator と Product の両方の階層設計が必要になり，設計が重くなる |


# Singletonパターン
*  Singleton（シングルトン）パターンは，特定クラスのインスタンスが一つだけ存在することを保証するデザインパターン
*  一般にインスタンスを作成すると，それを格納するためのメモリ領域を確保する必要があるので，同じクラスからのインスタンスが何個も必要ない場合には，このパターンを用いて，メモリの無駄遣いを防ぐ
*  Singletonパターンは，システム全体で共有されるリソースや設定を管理する場合などに使用される
>*  例えば，Factory Methodパターンで定義したConcreteCreatorのインスタンスは，誰が使っても同じなので，インスタンスは一つあれば十分である
>*  このような場合に，Singletonパターンが用いられる
*  その他の利用例：データベースの接続，設定情報の管理，ログの記録など

## すべてのクラスのスーパークラス`Object`
*  Pythonでは，自作クラスも既存のクラスもすべて`Object`クラスのサブクラスとして定義されている
*  任意のクラスのスーパークラスを調べたい場合は，そのクラスに定義されている`__bases__`属性（変数）を参照すればよい

**書式:** `クラス名.__base__`

In [15]:
class MyClass:
    pass

print(MyClass.__base__)

<class 'object'>


## `__new__`メソッド
*  `__new__`メソッドは特殊メソッドの一つで，クラス自身を引数として，インスタンス生成前に必ず呼び出されるクラスメソッド
*  クラスメソッドについては，[第5回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook05.ipynb)を参照
*  `__new__`メソッドは，`__init__`メソッド（コンストラクタ）よりも先に呼び出される
*  `__new__`メソッドは，クラスのインスタンスを生成して返す役割を持つ
>*  インスタンスを返さない（インスタンスが生成されない）と`__init__`メソッドが実行されない
>*  つまり，`__new__`メソッドでインスタンスを生成してから，`__init__`メソッドで属性の初期化が行われる
*  `__new__`メソッドは，Singletonパターンの実装などでよく使われる ⇒ 具体的には，`Object`クラス（スーパークラス）の`__new__`メソッドをオーバーライドする
*  次のサンプルコードにおける`super().__new__(cls)`は，スーパークラス（つまり`Object`クラス）の`__new__`メソッドを呼び出している
*  `__new__`メソッドの戻り値の型ヒントは「MyClass」となるが，[第6回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook06.ipynb)でも述べているように，クラスの定義が完了する前にそのクラス名を型ヒントとして使用すると例外や警告が発生するため，文字列「'MyClass'」で指定している

In [ ]:
# インスタンスを返すケース
class MyClass:
    def __new__(cls) -> 'MyClass':
        print('__new__の処理')
        return super().__new__(cls)

    def __init__(self) -> None:
        print('__init__の処理')

obj: MyClass = MyClass()
print(type(obj))

__new__の処理
__init__の処理
<class '__main__.MyClass'>


In [ ]:
# インスタンスを返さないケース
class MyClass:
    def __new__(cls) -> None:
        print('__new__の処理')

    def __init__(self) -> None:
        print('__init__の処理')

obj: None = MyClass() # インスタンスが生成されない ⇒ Noneとなる
print(type(obj))
print(obj)

__new__の処理
<class 'NoneType'>
None


## Singletonパターンの基本形
*  ただ一つのインスタンスしか生成できないクラス（`Singleton`クラス）を定義
*  そのクラスのただ一つのインスタンスを保持するためのクラス属性`instance`を定義（クラス属性・インスタンス属性については[第4回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook04.ipynb)を参照）
*  クラス属性`instance`の初期値は`None`にしておく ⇒ インスタンス化されるまでは何も格納されない
*  クラスメソッドである`__new__`メソッドを，ただ一つのインスタンスを生成するためのメソッドにオーバーライドする（オーバーライドについては[第4回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook04.ipynb)を参照）
*  `Singleton`クラスを継承したクラスのインスタンスも一つだけしか存在しない

## Singletonパターンの例 その1
*  以下のコードでは2つのインスタンス`a`と`b`を作成しており，それらが違うインスタンスであることを `is`演算子で確かめている
*  `is`演算子については，[第6回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook06.ipynb)を参照

In [ ]:
class DataBase:
    pass

a: DataBase = DataBase()
b: DataBase = DataBase()
print(a is b)

False


*  上のコードにSingletonパターンを適用すると以下のようになる
*  可視性をプライベートとしたクラス属性`__instance`は，インスタンスが生成されていない場合は`None`で，インスタンス化されると，そのインスタンスが格納される
*  そのため型ヒントは，そのクラス自身`Self`，または`None`を表す `Optional[Self]` となる
*  typyingモジュールの `Self` は「このクラス自身」を表す型ヒントとして使う
*  2回目以降のインスタンス化では，`__new__`メソッドの`if`文が`False`となり，既にあるインスタンスが返される
*  また，以下のコードでは，`Singleton`クラスを継承した様々なクラスにも適用できるように，`__new__`メソッドに可変長引数`*args`, `**kwargs`を定義している（可変長引数については[第3回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook03.ipynb)を参照）

In [34]:
from typing import Optional, Self

class Singleton:
    
    __instance: Optional[Self] = None

    def __new__(cls, *args, **kwargs) -> Self: # __new__のオーバーライド
        if cls.__instance is None:
            cls.__instance = super().__new__(cls)
        return cls.__instance

a: Singleton = Singleton()
b: Singleton = Singleton()
print(a is b)

True


## 多重継承
*  ここでは，次の例で利用される「多重継承」について説明する
*  多重継承とは，一つのクラスが複数のスーパークラスを継承すること（継承については[第4回講義資料](https://colab.research.google.com/github/yoshida-nu/lec_systemdesign/blob/main/doc/SystemDesign_notebook04.ipynb)を参照）
*  多重継承を使用すると，複数のクラスからが持つ特性（属性やメソッド）を組み合わせて新しいクラスを作成できる
*  しかし，過度に使用するとクラス間の依存関係が複雑になり，コードの理解と保守が難しくなる可能性があるため注意が必要

### 多重継承の例:
*  以下のコードにおいて，`Bird`クラスは`Animal`クラスと`FlyingObject`クラスをスーパークラスとして多重継承している
*  `Bird`クラスは`sayhello`メソッド（`Animal`クラスから継承）と `fly`メソッド（`FlyingObject`クラスから継承）を持っている
  
> <img src='./fig/multiple_inheritance_example.png' width='300'>

In [ ]:
# スーパークラス1
class Animal:
    
    def sayhello(self) -> None:
        pass

# スーパークラス2
class FlyingObject:
    
    def fly(self) -> None:
        pass

# サブクラス（多重継承）
class Bird(Animal, FlyingObject):
    
    def sayhello(self) -> None:
        print('Chirp, chirp!')

    def fly(self) -> None:
        print('I can fly!')

# Birdクラスのインスタンス作成
bird = Bird()
bird.sayhello()
bird.fly()

## Singletonパターンの例 その2
* 前述した「Factory Methodパターンの例 その1」のコードにSingletonパターンを適用する
* この例において，車の工場（`CarFactory`）や船の工場（`ShipFactory`）は，1つあればいいので，これらのクラスにSingletonパターンを適用する

In [39]:
from abc import ABC, abstractmethod
from typing import Optional, Self

# Product
class Product(ABC):

    def __init__(self, owner: str) -> None:
        self._owner = owner

    @abstractmethod
    def use(self) -> None:
        pass

# ConcreteProduct
class Car(Product):

    def use(self) -> None:
        print(f'{self._owner} is driving the car.')

# ConcreteProduct
class Ship(Product):

    def use(self) -> None:
        print(f'{self._owner} is driving the ship.')

# Creator
class Factory(ABC):

    @abstractmethod
    def create(self, owner: str) -> Product:
        pass

# Singleton
class Singleton:
    __instance: Optional[Self] = None

    def __new__(cls, *args, **kwargs) -> Self:
        if cls.__instance is None:
            cls.__instance = super().__new__(cls)
        return cls.__instance

# ConcreteCreator
class CarFactory(Factory, Singleton):

    def create(self, owner: str) -> Car:
        return Car(owner)

# ConcreteCreator
class ShipFactory(Factory, Singleton):

    def create(self, owner: str) -> Ship:
        return Ship(owner)

# クライアント
def client_code(factory: Factory, owner: str) -> None:
    product: Product = factory.create(owner)
    product.use()

car_factory: Factory = CarFactory()
ship_factory: Factory = ShipFactory()

client_code(car_factory, 'Alice')
client_code(car_factory, 'Bob')

client_code(ship_factory, 'John')
client_code(ship_factory, 'Paul')

# Factoryを2つ生成
car_factory2: Factory = CarFactory()
ship_factory2: Factory = ShipFactory()
print(car_factory is car_factory2)
print(ship_factory is ship_factory2)

Alice is driving the car.
Bob is driving the car.
John is driving the ship.
Paul is driving the ship.
True
True


## Singletonパターンのメリット・デメリット

| 観点           | メリット                                         | デメリット                                    |
| ------------ | -------------------------------------------- | ---------------------------------------- |
| **リソース** | インスタンスを 1つに制限できリソースの無駄遣いが防げる | 実質的にはグローバル変数となるので，無秩序に使うと依存が増えることになる   |
| **利便性** | 唯一のインスタンスにどこからでもアクセスできる ⇒ 利便性が高い             | どこからでもアクセスできるが逆に依存関係が強くなる      |



# インターフェース分離の原則（ISP: Interface Segregation Principle）

## ISPとは
*  システムを設計する際には，使っていないもの（メソッドなど）への依存を回避すべきだという原則
>*  つまり，クライアント（利用者）が利用しないメソッドへの依存を強制してはならないという原則
*  インタフェースに必要のないメソッドを追加して，継承先が無駄なコードを作成することがないようにすべき
*  インターフェースの持つメソッドは必要最小限のものにすべき ⇒ 小さく特定の役割に特化したインターフェースは，再利用しやすい
*  インターフェースの持つメソッドを増やしすぎると…
>*  継承先のクラスに無駄なコードが増え，コードの複雑化を引き起こす
>*  インターフェースの継承先が増えるため，インターフェースの修正による，継承先クラスの修正量が多くなり保守性が低下する
*  ISPに違反することによる影響:
>*  不要な依存関係が生じることで，クラス間の依存関係が強くなる
>*  インターフェース（抽象クラス）の機能が多くなるので，SRPに違反する可能性が高い
>*  結果的にコードの変更が難しくなる⇒保守性の低下
*  ISPに違反したコードを改善するために，インターフェース（抽象クラス）を分割する ⇒ 多重継承や集約（コンポジション）を使う

## ISPの具体例

### ISPを違反した例
*  以下のコードでは，多機能プリンタに関連する処理（メソッド）を定義するインターフェース`MultiFunctionPrinter`を定義している
*  このインターフェースを使用すると，プリント機能のみの単機能プリンタやスキャナーのような機器でも，このインターフェースの中のすべてのメソッドを実装する必要がある
*  これは非効率的で，ISPに違反している
*  ISPに違反している理由:
>*  `SimplePrinter`や`SimpleScanner`は，`MultiFunctionPrinter`を継承している（依存関係がある）
>*  このとき，`SimplePrinter`には必要のない`scan_document`メソッド等の実装を行う必要がある⇒使ってないものへの依存を強制している
>*  `SimpleScanner`も同様

> <img src='./fig/SOLID_ISP_sample1.png' width='430'>

In [ ]:
from abc import ABC, abstractmethod

# 抽象クラス（インターフェース）
class MultiFunctionPrinter(ABC):

    @abstractmethod
    def print_document(self) -> None:
        pass

    @abstractmethod
    def scan_document(self) -> None:
        pass

    @abstractmethod
    def fax_document(self) -> None:
        pass

    @abstractmethod
    def copy_document(self) -> None:
        pass

# 印刷機能のみを持つ
class SimplePrinter(MultiFunctionPrinter):
    
    def print_document(self) -> None:
        print('ドキュメントを印刷')

    # 使用しない機能も実装しなければならない
    def scan_document(self) -> None:
        print('スキャン機能はありません')

    def fax_document(self) -> None:
        print('Fax機能はありません')

    def copy_document(self) -> None:
        print('コピー機能はありません')

# スキャナ機能のみを持つ
class SimpleScanner(MultiFunctionPrinter):
    
    def print_document(self) -> None:
        print('印刷機能はありません')

    def scan_document(self) -> None:
        print('ドキュメントをスキャン')

    def fax_document(self) -> None:
        print('Fax機能はありません')

    def copy_document(self) -> None:
        print('コピー機能はありません')

# クライアント
printer = SimplePrinter()
printer.print_document()

scanner = SimpleScanner()
scanner.scan_document()

### 多重継承を使ってISPを遵守したコードに改善
*  ISPに従うために，インターフェースを分割する
*  具体的には，それぞれの機能（印刷，スキャン，ファックス，コピー）に対して独立したインターフェースを定義し，必要な機能を持つインターフェースだけを多重継承する
>*  `SimplePrinter`クラス: 印刷機能のみを持つ
>*  `SimpleScanner`クラス: スキャン機能のみを持つ
>*  `PrinterScanner`クラス: 印刷機能とスキャナ機能を持つ
*  これにより，単機能の機器や特定の機能のみを提供する機器に対して，必要なインターフェースのみを実装するだけでよくなり，余計な実装をしなくてすむ
*  また，多重継承を使うと（後述の集約を使ったコードより）シンプルで理解しやすい構造にすることができる

> <img src='./fig/SOLID_ISP_sample2.jpg' width='550'>

In [ ]:
from abc import ABC, abstractmethod

class Printer(ABC):
    
    @abstractmethod
    def print_document(self) -> None:
        pass

class Scanner(ABC):
    
    @abstractmethod
    def scan_document(self) -> None:
        pass

class Fax(ABC):
    
    @abstractmethod
    def fax_document(self) -> None:
        pass

class Copier(ABC):
    
    @abstractmethod
    def copy_document(self) -> None:
        pass

# 印刷機能のみを持つ
class SimplePrinter(Printer):
    
    def print_document(self) -> None:
        print('ドキュメントを印刷')

# スキャナ機能のみを持つ
class SimpleScanner(Scanner):
    
    def scan_document(self) -> None:
        print('ドキュメントをスキャン')

# 印刷機能とスキャナ機能を持つ
class PrinterScanner(Printer, Scanner): #多重継承
    
    def print_document(self) -> None:
        print('ドキュメントを印刷')

    def scan_document(self) -> None:
        print('ドキュメントをスキャン')

# クライアント
printer = SimplePrinter()
printer.print_document()
print('-------------------------------')
scanner = SimpleScanner()
scanner.scan_document()
print('-------------------------------')
printer_scanner = PrinterScanner()
printer_scanner.print_document()
printer_scanner.scan_document()

### 集約（コンポジション）を使ってISPを遵守したコードに改善（1）
*  上のコードと同様に，それぞれの機能（印刷，スキャン，ファックス，コピー）に対して独立したインターフェースを定義する ⇒ `Printer`, `Scanner`, `Fax`, `Copier`
*  各インターフェースの実装を定義するクラスをそれぞれ定義する ⇒ `BasicPrinter`, `BasicScanner`, `BasicFax`, `BasicCopier`
*  この例にはないが，各インターフェースの実装は複数定義することもできる
>*  例えば，より高速な印刷やスキャンを行う「`HighSpeed​​Printer`」や「`HighSpeed​​Scanner`」などを定義することもできる
*  上記のクラスのオブジェクトの一部を部分として持つクラスを定義する ⇒ `SimplePrinter`, `SimpleScanner`, `PrinterScanner`
*  このクラスをインスタンス化する際に，インスタンス属性となるインターフェースの実装を定義するクラスのオブジェクト（`BasicPrinter`, `BasicScanner`, `BasicFax`, `BasicCopier`）を引数として指定する
*  このとき指定する引数は抽象に依存した形になっている
*  また，`BasicPrinter`, `BasicScanner`, `BasicFax`, `BasicCopier`は，StrategyパターンにおけるStrategyと解釈することもできる
*  上述した`HighSpeed​​Printer`や`HighSpeed​​Scanner`などといった別のStrategyを定義することで，Contentの役割を持つ`SimplePrinter`, `SimpleScanner`, `PrinterScanner`がStrategyを選択できるようになる

<img src='./fig/SOLID_ISP_sample3.jpg' width='550'>

In [ ]:
from abc import ABC, abstractmethod

class Printer(ABC):
    
    @abstractmethod
    def print_document(self) -> None:
        pass

class Scanner(ABC):
    
    @abstractmethod
    def scan_document(self) -> None:
        pass

class Fax(ABC):
    
    @abstractmethod
    def fax_document(self) -> None:
        pass

class Copier(ABC):
    
    @abstractmethod
    def copy_document(self) -> None:
        pass

class BasicPrinter(Printer):
    
    def print_document(self) -> None:
        print('ドキュメントを印刷')

class BasicScanner(Scanner):
    
    def scan_document(self) -> None:
        print('ドキュメントをスキャン')

class BasicFax(Fax):
    
    def fax_document(self) -> None:
        print('ドキュメントをFaxで送信')

class BasicCopier(Copier):
    
    def copy_document(self) -> None:
        print('ドキュメントをコピー')
    
# 印刷機能のみを持つ
class SimplePrinter:
    
    def __init__(self, printer: Printer) -> None:
        self.printer = printer
    
    def execute_print(self) -> None:
        self.printer.print_document()

# スキャナ機能のみを持つ
class SimpleScanner:
    
    def __init__(self, scanner: Scanner) -> None:
        self.scanner = scanner
    
    def execute_scan(self) -> None:
        self.scanner.scan_document()

# 印刷機能とスキャナ機能を持つ
class PrinterScanner:
    
    def __init__(self, printer: Printer, scanner: Scanner) -> None:
        self.printer = printer
        self.scanner = scanner
    
    def execute_print(self) -> None:
        self.printer.print_document()

    def execute_scan(self) -> None:
        self.scanner.scan_document()

# クライアント
printer = SimplePrinter(BasicPrinter())
printer.execute_print()
print('-------------------------------')
scanner = SimpleScanner(BasicScanner())
scanner.execute_scan()
print('-------------------------------')
printer_scanner = PrinterScanner(BasicPrinter(), BasicScanner())
printer_scanner.execute_print()
printer_scanner.execute_scan()

### 集約（コンポジション）を使ってISPを遵守したコードに改善（2）
*  上のコードにおける，`SimplePrinter`, `SimpleScanner`, `PrinterScanner`を一つクラスとしてまとめることを考える
*  異なる機能を持つデバイスを柔軟に構成するための`MultiFunctionDevice`クラスを新たに定義
*  `MultiFunctionDevice`クラスをインスタンス化する際に渡す引数によって，様々なバリエーションのデバイスが表現できる
>*  印刷機能のみを持つデバイス: `MultiFunctionDevice(printer = BasicPrinter())`
>*  スキャン機能のみを持つデバイス: `MultiFunctionDevice(scanner = BasicScanner())`
>*  印刷機能とスキャナ機能を持つデバイス: `MultiFunctionDevice(printer = BasicPrinter(), scanner = BasicScanner())`
*  引数はすべてデフォルト値を`None`としているので，必要な機能のみを指定すればよい
*  `MultiFunctionDevice`クラスの各メソッド（`print_document`や`scan_document`など）は，該当するオブジェクトが設定されていない（属性が`None`である）場合に，その機能が使えないといったメッセージを表示する
*  このように設計することで，オブジェクトが未設定のために生じる不具合を回避できる
*  ただし，この設計（コード）では，クライアントからみると必要のないメソッドを公開している（正常に呼び出せる）ので，例外を発生させて例外として処理をしたほうが望ましいともいえる

> <img src='./fig/SOLID_ISP_sample4.jpg' width='600'>

In [ ]:
from typing import Optional
from abc import ABC, abstractmethod

class Printer(ABC):
    
    @abstractmethod
    def print_document(self) -> None:
        pass

class Scanner(ABC):
    
    @abstractmethod
    def scan_document(self) -> None:
        pass

class Fax(ABC):
    
    @abstractmethod
    def fax_document(self) -> None:
        pass

class Copier(ABC):
    
    @abstractmethod
    def copy_document(self) -> None:
        pass

class BasicPrinter(Printer):
    
    def print_document(self) -> None:
        print('ドキュメントを印刷')

class BasicScanner(Scanner):
    
    def scan_document(self) -> None:
        print('ドキュメントをスキャン')

class BasicFax(Fax):
    
    def fax_document(self) -> None:
        print('ドキュメントをFaxで送信')

class BasicCopier(Copier):
    
    def copy_document(self) -> None:
        print('ドキュメントをコピー')

class MultiFunctionDevice:
    
    def __init__(self, printer: Optional[Printer] = None, scanner: Optional[Scanner] = None,
                 fax: Optional[Fax] = None, copier: Optional[Copier] = None) -> None:
        self.printer = printer
        self.scanner = scanner
        self.fax = fax
        self.copier = copier

    def execute_print(self) -> None:
        if self.printer:
            self.printer.print_document()
        else:
            print('印刷機能はありません')

    def execute_scan(self) -> None:
        if self.scanner:
            self.scanner.scan_document()
        else:
            print('スキャン機能はありません')

    def execute_fax(self) -> None:
        if self.fax:
            self.fax.fax_document()
        else:
            print('Fax機能はありません')

    def execute_copy(self) -> None:
        if self.copier:
            self.copier.copy_document()
        else:
            print('コピー機能はありません')

# 印刷機能のみを持つデバイス
simple_printer = MultiFunctionDevice(printer = BasicPrinter())
simple_printer.execute_print()
simple_printer.execute_scan()
print('-------------------------------')
# スキャナ機能のみを持つデバイス
simple_scanner = MultiFunctionDevice(scanner = BasicScanner())
simple_scanner.execute_print()
simple_scanner.execute_scan()
print('-------------------------------')
# 印刷機能とスキャナ機能を持つデバイス
printer_scanner = MultiFunctionDevice(printer = BasicPrinter(), scanner = BasicScanner())
printer_scanner.execute_print()
printer_scanner.execute_scan()

## ISPを遵守することによる効果
*  小さなインターフェースを設計することで，クライアントは必要なメソッドのみを利用できるようになる
>*  一つのインターフェースに多くの機能（メソッド）が含まれている場合，クライアントがそのインターフェースの一部だけを必要とする場合でも，全体を実装する必要があり，不必要なメソッドの変更が他に影響を与える可能性がある
*  必要なメソッドだけに依存するため，システムを拡張する際に他のクラスやインターフェースに影響を与えにくくなる
*  各インターフェースが単一の責任に絞られる（ISPはインターフェースに対するSRPともいえる）
*  インターフェースが小さくなるため，変更が必要な場合でも，影響範囲が限定される
>*  大きなインターフェースを変更する場合，そのインターフェースを実装しているすべてのクラスに影響を与える可能性がある
>*  一方，小さなインターフェースを使用していれば，その影響は限定的になる
*  クライアントは必要なインターフェースにのみ依存するため，システム全体の依存関係が簡素化される
*  ISPを遵守することで設計の手間は増えるかもしれないが，基本的にはメリットのほうが大きい

## ISPとフールプルーフ
*  ISPにより，利用者が操作できる範囲を制限し，誤操作を防ぐことができる
*  例えば，爆破ボタン付きのスマートフォンを開発し，利用者に「このボタンは押さないでください」といっても，間違って押してしまう利用者がいるかもしれないので，そのようなボタンは最初から作らないほうがよい
*  ISPにより，インターフェースをシンプルにするができる ⇒ その結果，利用者の誤解を防ぎ，安全に操作できる設計を実現できる

# 依存性逆転の原則（DIP: Dependency Inversion Principle）


## DIPとは
*  コードの依存関係が抽象だけを参照している
>*  抽象とは，抽象クラスやインターフェースと解釈すればよい
*  上位のモジュール（クラス）は下位のモジュール（クラス）に直接依存すべきではなく，両者は抽象に依存すべきであるという原則
*  抽象に依存する⇒具象に依存しない
>*  変化しやすい具象クラスを参照・継承しない
>*  具象メソッドをオーバーライドしない

## 上位モジュールと下位モジュールの例
*  ネットショップにおける注文処理を上位モジュールとする
*  上位モジュールは注文処理を行うという目的を持つ
*  注文処理モジュールの下位モジュールは，上位モジュールの目的を達成するための手段に対応
*  注文処理を実現するためには，例えば，ユーザが存在するかどうか（ユーザの有効性）のチェック，注文された商品の在庫があるかのチェック，支払い処理，在庫情報の更新などの処理が必要となる
*  この例の場合の下位モジュール：
>*  ユーザの有効性をチェックするモジュール
>*  商品の在庫をチェックするモジュール
>*  支払い処理を行うモジュール
>*  在庫情報の更新処理を行うモジュール
*  この例からもイメージできるように，上位モジュールの目的を実現するために，複数の小さな処理（手段）に分割したものが下位モジュールとなる
*  したがって，この考え方では，上位モジュールが下位モジュールに依存することになる
*  つまり，手段の変更によって目的を変えなければならないといった問題が発生する
*  この依存関係を解消する（依存の方向性を逆転する）必要があるという原則がDIPとなる

## DIPを遵守するための方法
*  インターフェースや抽象クラスを使用する
>*  上位モジュールと下位モジュールを，共通のインターフェースまたは抽象クラスに依存させる
>*  これにより，上位モジュールは，下位モジュールの実装を直接参照しないため，依存性が逆転する
*  Dependency Injection（DI: 依存性注入）パターンを使用する
>*  DIパターンは，デザインパターンの一つ（GoFのデザインパターンではない）
>*  GoFのデザインパターンではないので，ソフトウェア設計における手法・原則として位置づけられることもある
>*  必要な下位モジュールを直接作成せずに外部から提供する
>*  これにより，上位モジュールが具体的な下位モジュールに依存しなくなる

## DIPの具体例

### DIPを違反している例
*  以下のコードでは，`EmailService`クラスと`Notification`クラスを定義している  
  
*  `EmailService`クラス:
>*  具体的な通知方法（メール）を定義するクラス（下位モジュール）
>*  任意のメッセージ`msg`をメールで送信する`send_email`メソッドが定義されている
* `Notification`クラス:
>*  個々の通知を表すためのクラス（上位モジュール）
>*  多数の利用者に対して様々な通知を送るシステムをイメージしている
>*  通知サービスオブジェクト`service`として，`EmailService`クラスのオブジェクトが用いられており，任意のメッセージ`msg`を`send_email`メソッドを呼び出して通知する`send`メソッドが定義されている
>*  つまり，`Notification`クラスは，メッセージを通知するためのクラスであるが，通知方法が具体的な方法である`EmailService`クラス（具象クラス）の`send_email`メソッド（具象メソッド）に依存している ⇒ DIPに違反

> <img src='./fig/SOLID_DIP_sample1.jpg' width='500'>  

*  `Notification`クラスと`EmailService`クラスの間には依存関係（メソッドを呼び出す側と呼び出される側という一時的な関係）とコンポジションの関連がある

In [ ]:
class EmailService:
    
    def send_email(self, msg: str) -> None:
        print(f'メール送信: {msg}')

class Notification:
    
    def __init__(self) -> None:
        self.__service = EmailService()

    def send(self, msg: str) -> None:
        self.__service.send_email(msg)

notification = Notification()
notification.send('Hello!')

### DIPを違反していることによる影響
*  上のコードで，通知方法をメールから別の方法に変更する場合には，新たな通知方法（SMSやプッシュ通知など）を定義したクラスを追加するだけでなく，`Notification`クラスを変更する必要もある（下のクラス図とコード参照）
*  これは非効率的であり，DIPに違反している（具体的なクラスやメソッドを参照している）

> <img src='./fig/SOLID_DIP_sample2.jpg' width='520'>  

In [ ]:
class EmailService:
    
    def send_email(self, msg: str) -> None:
        print(f'メール送信: {msg}')

# 新たな通知方法
class ShortMessageService:
    
    def send_sms(self, msg: str) -> None:
        print(f'SMS送信: {msg}')

# 通知方法を変更するには，この中も変更する必要がある
class Notification:
    
    def __init__(self) -> None:
        self.__service = ShortMessageService() # 変更箇所

    def send(self, msg: str) -> None:
        self.__service.send_sms(msg)

notification = Notification()
notification.send('Hello!')

### DIPを遵守したコードに改良
*  メッセージ通知のための抽象的な`MessageService`インターフェース（上位モジュール）を定義
*  `MessageService`で抽象メソッド`send_message`を定義
*  `MessageService`インターフェースを実装する具体的なクラス（`MessageService`を継承したクラス）である`EmailService`クラス（下位モジュール）を定義
*  `EmailService`で`send_message`メソッドの実装を定義
*  `Notification`クラス（上位モジュール）の`service`属性は，型ヒントを用いて抽象（`MessageService`）に依存させた形にする
*  `send`メソッドも抽象メソッドである`send_message`に依存させる  

> <img src='./fig/SOLID_DIP_sample3.jpg' width='520'>  

In [ ]:
from abc import ABC, abstractmethod

class MessageService(ABC):
    
    @abstractmethod
    def send_message(self, msg: str) -> None:
        pass

class EmailService(MessageService):
    
    def send_message(self, msg: str) -> None:
        print(f'メール送信: {msg}')

class Notification:
    
    def __init__(self, service: MessageService) -> None: #引数を抽象（MessageService）に依存させる
        self.__service = service

    def send(self, msg: str) -> None:
        self.__service.send_message(msg) #抽象メソッドに依存（実装はEmailserviceで定義）

notification = Notification(EmailService())
notification.send('Hello!')

### DIPを遵守していることによる効果
*  インターフェースを用いることにより，`Notification`クラスを，抽象的な`MessageService`インターフェースに依存させた形にできた
*  これにより，具体的な通知方法（具象メソッド）に直接依存することがなくなり，拡張性と柔軟性が高い設計が可能となる  
*  例えば，新たな通知方法としてSMSを追加する場合は，具体的な通知方法（`ShortMessageService`クラス）を追加するだけで済む

> <img src='./fig/SOLID_DIP_sample4.jpg' width='580'>  

In [ ]:
from abc import ABC, abstractmethod

class MessageService(ABC):
    
    @abstractmethod
    def send_message(self, msg: str) -> None:
        pass

class EmailService(MessageService):
    
    def send_message(self, msg: str) -> None:
        print(f'メール送信: {msg}')

#他の部分の変更なしでクラスを追加するだけでよい
class ShortMessageService(MessageService):
    
    def send_message(self, msg: str) -> None:
        print(f'SMS送信: {msg}')

class Notification:
    
    def __init__(self, service: MessageService) -> None:
        self.__service = service

    def send(self, msg: str) -> None:
        self.__service.send_message(msg)

notification_email = Notification(EmailService())
notification_email.send('こんにちは！')
notification_sms = Notification(ShortMessageService())
notification_sms.send('Hello!')

## DIPを遵守することによる効果
*  下位モジュールの具体的な実装を変更しても，上位モジュールや他の部分に影響を与えない
*  上位モジュールを異なる環境や状況で再利用しやすくなる
*  下位モジュールの実装が未完成な場合でも，テスト用の簡易的なモジュール使ったテストが実行可能
*  新しい下位モジュールを追加しても，上位モジュールを変更する必要がない ⇒ OCPにも貢献
*  つまり，DIPを遵守することで，変更と拡張に強く，テストしやすいコードになる

# 参考資料
*  Robert C.Martin (著), 角征典 (翻訳), 高木正弘 (翻訳), [Clean Architecture 達人に学ぶソフトウェアの構造と設計](https://www.kadokawa.co.jp/product/301806000678/), KADOKAWA, 2018.
*  ひらまつしょうたろう, [Python で身につける オブジェクト指向【SOLID原則+デザインパターンで、オブジェクト指向設計 の基礎を習得！】](https://www.udemy.com/course/python-solid-design-pattern/?couponCode=KEEPLEARNING), Udemy, 最終更新日 2024/6.
*  Mark Summerfield (著), 斎藤康毅 (訳), [実践 Python 3](https://www.oreilly.co.jp/books/9784873117393/), オライリージャパン, 2015.
<!-- *  Bill Lubanovic (著), 鈴木駿 (監訳), 長尾高弘 (訳), [入門 Python 3 第2版](https://www.oreilly.co.jp/books/9784873119328/), オライリージャパン, 2021. -->
<!-- *  Guido van Rossum (著), 鴨澤眞夫 (翻訳), [Pythonチュートリアル 第4版](https://www.oreilly.co.jp/books/9784873119359/), オライリージャパン, 2021. -->

